In [1]:
import os

os.environ["OPENAI_API_BASE"]='http://localhost:8081/v1'

os.environ["OPENAI_MODEL_NAME"]='gpt-4o'
# custom_model_name from the bash script

os.environ["OPENAI_API_KEY"] = "MGHA8LSIAn1QDDUPb1s722NBhlg%2BohupOR5P%2FNRLugvzmgsLFHhaudt5YFstWbT74367Z0Q5hcPisFLacAvO4yefKFAOxXz88vWYLoeY0ktY8%2FokKUOe4P0NwNRe9t38kU0fgZcHUVYBGZdNmAAC64b3%2BmBx24QMljhr0d9pHn4%3D_1712555524044r4cj5g9ab0g"

In [3]:
from crewai import Agent

database_specialist_agent = Agent(
  role = "Database specialist",
  goal = "Provide data to answer business questions using SQL",
  backstory = '''You are an expert in SQL, so you can help the team
  to gather needed data to power their decisions.
  You are very accurate and take into account all the nuances in data.''',
  allow_delegation = False,
  verbose = True
)

tech_writer_agent = Agent(
  role = "Technical writer",
  goal = '''Write engaging and factually accurate technical documentation
    for data sources or tools''',
  backstory = '''
  You are an expert in both technology and communications, so you can easily explain even sophisticated concepts.
  You base your work on the factual information provided by your colleagues.
  Your texts are concise and can be easily understood by a wide audience.
  You use professional but rather an informal style in your communication.
  ''',
  allow_delegation = False,
  verbose = True
)

In [4]:
MY_HOST = '127.0.0.1' # default address
MY_USER = "root"
MY_PWD = "123456"

import pymysql
import json
from datetime import datetime

class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super(DateTimeEncoder, self).default(obj)

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = pymysql.connect(
            host=host_name,
            user=user_name,
            password=user_password,
        )
        print("Connection to MySQL DB successful")
    except pymysql.MySQLError as e:
        print(f"The error '{e}' occurred")
    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        result = cursor.fetchall()
        field_names = [i[0] for i in cursor.description]
        result = [dict(zip(field_names, row)) for row in result]
        str = json.dumps(result, indent=4, cls=DateTimeEncoder)
        return str
    except pymysql.MySQLError as e:
        print(f"The error '{e}' occurred")

conn = create_connection(MY_HOST, MY_USER, MY_PWD)
# result = execute_query(conn, "SHOW CREATE TABLE ecommerce_db.sessions;")
# print(result)

Connection to MySQL DB successful


In [5]:
from crewai_tools import BaseTool

class DatabaseQuery(BaseTool):
  name: str = "Database Query"
  description: str = "Returns the result of SQL query execution"

  def _run(self, sql_query: str) -> str:
      # Implementation goes here
      return get_clickhouse_data(sql_query)

database_query_tool = DatabaseQuery()

In [14]:
from crewai import Task
from crewai.tasks.task_output import TaskOutput
import time

def callback_function(output: TaskOutput):
    # Do something after the task is completed
    # Example: Send an email to the manager
    time.sleep(8)
    print(f"""
        Task completed!
        Task: {output.description}
        Output: {output.raw_output}
    """)

tech_writer_agent = Agent(
  role = "Technical writer",
  goal = '''Write engaging and factually accurate technical documentation 
  for data sources or tools''',
  backstory = ''' 
  You are an expert in both technology and communications, so you 
  can easily explain even sophisticated concepts.
  Your texts are concise and can be easily understood by wide audience. 
  You use professional but rather informal style in your communication.
  You base your work on the factual information provided by your colleagues. 
  You stick to the facts in the documentation and use ONLY 
  information provided by the colleagues not adding anything.''',
  allow_delegation = False,
  verbose = True
)

table_documentation_task = Task(
  description = '''Using provided information about the table, 
  put together the detailed documentation for this table so that 
  people can use it in practice''',
  expected_output = '''Well-written detailed documentation describing 
  the data scheme for the table {table} in markdown format, 
  that gives the table overview in 1-2 sentences then then 
  describes each columm. Structure the columns description 
  as a markdown table with column name, type and description.
  The documentation is based ONLY on the information provided 
  by the database specialist without any additions.''',
  tools = [],
  output_file = "table_documentation.md",
  agent = tech_writer_agent,
  callback=callback_function
)

In [15]:
from crewai import Crew

crew = Crew(
  agents = [database_specialist_agent, tech_writer_agent],
  tasks = [table_description_task,  table_documentation_task],
  verbose = 2
)

result = crew.kickoff({'table': 'ecommerce_db.users'})

2024-06-29 15:12:40,752 - 140704597691264 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-06-29 15:12:40][DEBUG]: == Working Agent: Database specialist
 [2024-06-29 15:12:40][INFO]: == Starting Task: Provide the comprehensive overview for the data 
  in table ecommerce_db.users, so that it's easy to understand the structure 
  of the data. This task is crucial to put together the documentation 
  for our database


> Entering new CrewAgentExecutor chain...
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incremented
Formatting errors incr